## Processing Post-Harvest datasets

In [1]:
import pandas as pd

In [ ]:
def transform_data(sheets_dict: dict[pd.DataFrame], 
                   lookup: dict[str,str], 
                   file_origin: str) -> pd.DataFrame:
    # mapping from file_origin to rec_type letter
    rec_map = {
        "harvested_area": "H",
        "physical_area":  "A",
        "yield":          "Y",
        "production":     "P"
    }
    # mapping tech suffix → human label
    tech_map = {
        "A": "all technologies",
        "I": "irrigation",
        "R": "rainfed"
    }
    rec_code = rec_map[file_origin]
    
    long_dfs = []
    for sheet_name, df in sheets_dict.items():
        # 1) infer tech_type from sheet name suffix (_TA, _TI, _TR → A, I, R)
        if sheet_name.endswith("(TA)"):
            tech = "A"
        elif sheet_name.endswith("(TI)"):
            tech = "I"
        elif sheet_name.endswith("(TR)"):
            tech = "R"
        else:
            raise ValueError(f"Unrecognized tech suffix in '{sheet_name}'")
        
        # 2) Rename x/y → latitude/longitude
        df = df.rename(columns={
            "x": lookup["x"],
            "y": lookup["y"]
        })
        
        # 3) Rename each crop column: e.g. BANA_I → Banana_I
        rename_map = {}
        for col in df.columns:
            if col.upper().endswith(f"_{tech}"):
                prefix = col.rsplit("_", 1)[0].lower()  # e.g. "bana"
                full_crop = lookup.get(prefix)
                if full_crop is None:
                    # skip any non-crop or unexpected
                    continue
                rename_map[col] = f"{full_crop}_{tech}"
        df = df.rename(columns=rename_map)
        
        # 4) Melt to long: keep all non-crop columns as id_vars
        crop_cols = list(rename_map.values())
        id_vars = [c for c in df.columns if c not in crop_cols]
        df_long = df.melt(
            id_vars=id_vars,
            value_vars=crop_cols,
            var_name="variable",
            value_name="value"
        )
        
        # 5) Split “variable” → Crop type & tech_type
        df_long["tech_type"] = df_long["variable"].str[-1]
        df_long["Crop type"] = df_long["variable"].str[:-2]
        df_long = df_long.drop(columns=["variable"])
        
        # 6) Add water‐management regime & rec_type
        df_long["water‐management regime"] = df_long["tech_type"].map(tech_map)
        df_long["rec_type"] = rec_code
        
        # 7) Reorder & select final columns
        final_cols = [
            lookup["y"],              # latitude of pixel center
            lookup["x"],              # longitude of pixel center
            "Crop type",
            "tech_type",
            "water‐management regime",
            "rec_type",
            "ADM1_NAME",
            "ADM2_NAME",
            "unit",
            "grid_code",
            "year_data",
            "value"
        ]
        # (rename latitude/longitude to the exact strings)
        df_long = df_long[final_cols]
        
        long_dfs.append(df_long)
    
    # 8) Concatenate all tech-types for this file
    result = pd.concat(long_dfs, ignore_index=True)
    return result

# Example usage:
# import json
# lookup = json.load(open("lookup.json"))
# harvested_sheets = pd.read_excel("harvested_area.xlsx", sheet_name=None)
# ha_long = transform_data(harvested_sheets, lookup, "harvested_area")
# physical_sheets = pd.read_excel("physical_area.xlsx", sheet_name=None)
# pa_long = transform_data(physical_sheets, lookup, "physical_area")
# ... and similarly for yield & production




In [ ]:
import json
lookup = json.load(open("lookup.json"))
harvested_sheets = pd.read_excel("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/harvested_area.xlsx", sheet_name=None)
ha_long = transform_data(harvested_sheets, lookup, "harvested_area")
physical_sheets = pd.read_excel("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/Physical_area.xlsx", sheet_name=None)
pa_long = transform_data(physical_sheets, lookup, "physical_area")

In [17]:
lookup = json.load(open("lookup.json"))
Production_sheets = pd.read_excel("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/Production.xlsx", sheet_name=None)
Prod_long = transform_data(Production_sheets, lookup, "production")
yield_sheets = pd.read_excel("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/yield.xlsx", sheet_name=None)
yield_long = transform_data(yield_sheets, lookup, "yield")

In [24]:
# Save the transformed data to excel (XLSX) files
ha_long.to_csv("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed/harvested_long.xlsx", index=False)
pa_long.to_csv("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed/physical_long.xlsx", index=False)
yield_long.to_csv("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed/yield_long.xlsx", index=False)
Prod_long.to_csv("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed/production_long.xlsx", index=False)

In [14]:
# harvest= pd.read_csv("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed/harvested_long.csv")
physical= pd.read_csv("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed/physical_long.csv")
yield_data= pd.read_csv("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed/yield_long.csv")
production= pd.read_csv("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed/production_long.csv")

In [ ]:
physical.columns

Index(['longitude_of_pixel_center', 'latitude_of_pixel_center', 'crop_type',
       'tech_type', 'water_management_regime', 'rec_type', 'state', 'town',
       'unit', 'grid_code', 'year', 'value'],
      dtype='object')

In [39]:
# assume `df` is your long‐format DataFrame
state_summaries = (
    production
    .groupby([
        "state",     # the state/province
        "rec_type",      # H, A, Y, or P
        "crop_type",     # e.g. Maize, Rice
        "tech_type",     # A, I, or R
        'water_management_regime',
        "year"      # calendar year
    ], as_index=False)
    .agg(value_sum = ("value", "sum"))
).sort_values(
    by=["state", "crop_type", "tech_type", "rec_type", "year"]
)

In [41]:
state_summaries.to_excel("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed_agg/production.xlsx", index=False)

In [40]:
state_summaries

,state,rec_type,crop_type,tech_type,water_management_regime,year,value_sum
0,Abia,P,Arabic Coffee,A,all technologies,avg(2019-2021),0.0
1,Abia,P,Arabic Coffee,R,rainfed,avg(2019-2021),0.0
2,Abia,P,Banana,A,all technologies,avg(2019-2021),0.0
3,Abia,P,Banana,R,rainfed,avg(2019-2021),0.0
4,Abia,P,Barley,A,all technologies,avg(2019-2021),0.0
...,...,...,...,...,...,...,...
4733,Zamfara,P,Wheat,I,irrigation,avg(2019-2021),1122.3
4734,Zamfara,P,Wheat,R,rainfed,avg(2019-2021),184.8
4735,Zamfara,P,Yams,A,all technologies,avg(2019-2021),593726.5
4736,Zamfara,P,Yams,I,irrigation,avg(2019-2021),0.0


## Other datasets

In [1]:
import os
import pandas as pd

def extract_data(folder_path):
    extracted_data = {}
    suffix_counter = {}

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        base_name, ext = os.path.splitext(file_name)

        if ext.lower() in ['.csv', '.xlsx', '.xls']:
            try:
                if ext.lower() == '.csv':
                    # Single DataFrame for CSV
                    df_dict = {base_name: pd.read_csv(file_path)}
                else:
                    # Read all sheets into a dict of DataFrames
                    df_dict = pd.read_excel(file_path, sheet_name=None)
                    # Prefix sheet names with base filename
                    df_dict = {f"{base_name}_{sheet_name}": df
                               for sheet_name, df in df_dict.items()}

                for key, df in df_dict.items():
                    unique_key = key
                    # Handle key collisions
                    if unique_key in extracted_data:
                        # Initialize counter for this key if needed
                        if key not in suffix_counter:
                            suffix_counter[key] = 1
                        suffix_counter[key] += 1
                        unique_key = f"{key}_{suffix_counter[key]}"

                    extracted_data[unique_key] = df

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

    return extracted_data


In [2]:
#Specify full folder path to the dataset
df = extract_data("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset")

C:\Users\USER\AppData\Local\Temp\ipykernel_22596\4012688409.py:16: DtypeWarning: Columns (1,3,4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dict = {base_name: pd.read_csv(file_path)}
C:\Users\USER\AppData\Local\Temp\ipykernel_22596\4012688409.py:16: DtypeWarning: Columns (4,5,6,9,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dict = {base_name: pd.read_csv(file_path)}


In [3]:
df.keys()

dict_keys(['Data', 'FAOSTAT_data_en_5-8-2025', 'global-market-monitor_subnational', 'nga-rainfall-adm2-full', 'Population data', 'wfp_food_prices_nga'])

In [ ]:
logistics= df['Data']
food_price= df['wfp_food_prices_nga']
rainfall= df['nga-rainfall-adm2-full']
population= df['Population data']
market = df['global-market-monitor_subnational']
combined_metrics = df['FAOSTAT_data_en_5-8-2025']

In [5]:
logistics.columns

Index(['m49_code', 'country', 'region', 'cpc_code', 'commodity', 'year',
       'loss_percentage', 'loss_percentage_original', 'loss_quantity',
       'activity', 'food_supply_stage', 'treatment', 'cause_of_loss',
       'sample_size', 'method_data_collection', 'reference', 'url', 'notes'],
      dtype='object')

In [6]:
logistics=logistics[(logistics['year'] >= 2019) & (logistics['year'] <= 2021)][['m49_code', 'country', 'region', 'cpc_code', 'commodity', 'year',
        'loss_percentage', 'loss_percentage_original', 'loss_quantity',
        'activity', 'food_supply_stage', 'treatment', 'cause_of_loss',
        'sample_size', 'method_data_collection']]

In [7]:
logistics.fillna("N/A", inplace=True)

In [45]:
logistics.to_excel("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/processed/logistics.xlsx", index=False)

### Food Price

In [9]:
food_price.drop(index= 0, inplace= True)
food_price.rename(columns={
                'admin1': 'State',
                'admin2': 'town',
                'price': 'price(NGN)',
                'usdprice': 'price(USD)'
            }, inplace=True)
food_price['year']= pd.to_datetime(food_price['date']).dt.year
food_price = food_price[(food_price['year'] >= 2019) & (food_price['year'] <= 2021)]

In [46]:
food_price.to_excel("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/processed/food_price.xlsx", index=False)

### Market data

In [11]:
market.drop(index= 0, inplace= True)
market.rename(columns={
                'Admin1': 'State',
            }, inplace=True)
market['Year'] = pd.to_datetime(market['Date']).dt.year

#Handling missing values
market.fillna("N/A", inplace=True)

market= market[['Date', 'MonthlyVersion', 'FrequencyName',
       'State', 'MainStapleFood', 'PriceType',
       'CaloricContribution', 'QuarterlyChangeNSA', 'MonthlyChangeNSA',
       'QuarterlyChangeSA', 'MonthlyChangeSA', 'YoYChangeQuarter',
       'YoYChangeMonth', 'PriceTrendQuarter', 'PriceTrendMonth',
       'QuarterlyCostShare', 'TotImpactQuarterlyChange',
       'TotImpactQuarterlyCode', 'TotImpactMonthlyChange',
       'TotImpactMonthlyCode']]

In [47]:
market.to_excel("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/processed/market.xlsx", index=False)

In [13]:
market

,Date,MonthlyVersion,FrequencyName,State,MainStapleFood,PriceType,CaloricContribution,QuarterlyChangeNSA,MonthlyChangeNSA,QuarterlyChangeSA,MonthlyChangeSA,YoYChangeQuarter,YoYChangeMonth,PriceTrendQuarter,PriceTrendMonth,QuarterlyCostShare,TotImpactQuarterlyChange,TotImpactQuarterlyCode,TotImpactMonthlyChange,TotImpactMonthlyCode
1,3/1/2021,55,Monthly,Abia,Sorghum (white),Wholesale,13,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,21,N/A,N/A,N/A,N/A
2,3/1/2021,55,Monthly,Abia,Maize (white),Wholesale,8,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,7,N/A,N/A,N/A,N/A
3,3/1/2021,55,Monthly,Abia,Bread,Retail,5,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,3/1/2021,55,Monthly,Abia,Rice (imported),Wholesale,8,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,20,N/A,N/A,N/A,N/A
5,3/1/2021,55,Monthly,Abia,Gari (white),Wholesale,10,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,15,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908,6/1/2023,82,Monthly,Zamfara,Gari (white),Wholesale,10,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,,N/A,
2909,6/1/2023,82,Monthly,Zamfara,Sorghum (white),Wholesale,13,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,,N/A,
2910,6/1/2023,82,Monthly,Zamfara,Bread,Retail,5,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,,N/A,
2911,6/1/2023,82,Monthly,Zamfara,Maize (white),Wholesale,8,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,,N/A,


### Population

In [49]:
combined_metrics.to_excel("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/processed/combined_metrics.xlsx", index=False)

In [27]:
combined_metrics=combined_metrics[['Domain Code', 'Domain', 'Area Code (M49)', 'Area', 'Element Code',
       'Element', 'Item Code (FBS)', 'Item', 'Year', 'Unit',
       'Value', 'Flag', 'Flag Description']]

In [48]:
population.to_excel("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/processed/population.xlsx", index=False)

### Rainfall

In [54]:
# rainfall.drop(index= 0, inplace= True)
rainfall.rename(columns={
                'Admin1': 'State',
            }, inplace=True)
rainfall['Year'] = pd.to_datetime(rainfall['date']).dt.year

#Handling missing values
rainfall.fillna("N/A", inplace=True)

rainfall= rainfall[(rainfall['year'] >= 2019) & (rainfall['year'] <= 2021)][['date', 'adm2_id', 'ADM2_PCODE', 'n_pixels', 'rfh', 'rfh_avg', 'r1h',
       'r1h_avg', 'r3h', 'r3h_avg', 'rfq', 'r1q', 'r3q', 'version']]

KeyError: 'year'

In [50]:
rainfall.to_excel("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/processed/rainfall.xlsx", index=False)

ValueError: This sheet is too large! Your sheet size is: 1219345, 14 Max sheet size is: 1048576, 16384

In [51]:
rainfall

,date,adm2_id,ADM2_PCODE,n_pixels,rfh,rfh_avg,r1h,r1h_avg,r3h,r3h_avg,rfq,r1q,r3q,version
0,#date,#adm2+id,#adm2+code,#indicator+n_pixels+num,#indicator+rfh+num,#indicator+rfh_avg+num,#indicator+r1h+num,#indicator+r1h_avg+num,#indicator+r3h+num,#indicator+r3h_avg+num,#indicator+rfq+pct,#indicator+r1q+pct,#indicator+r3q+pct,#status
1,1981-01-01,22980,NG029014,28.0,4.3571,3.2274,NaN,NaN,NaN,NaN,113.7317,NaN,NaN,final
2,1981-01-11,22980,NG029014,28.0,2.75,3.7071,NaN,NaN,NaN,NaN,89.0074,NaN,NaN,final
3,1981-01-21,22980,NG029014,28.0,7.0714,6.0607,14.1786,12.9952,NaN,NaN,109.1379,105.146,NaN,final
4,1981-02-01,22980,NG029014,28.0,4.3214,9.3012,14.1429,19.069,NaN,NaN,65.1794,83.0535,NaN,final
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219340,2025-03-11,23001,NG030013,33.0,57.0909,33.0414,94.2424,68.3,116.4242,103.6051,163.2192,133.1321,111.284,final
1219341,2025-03-21,23001,NG030013,33.0,70.0909,36.7737,144.5758,93.3232,185.5152,137.9192,179.7563,149.6041,132.177,final
1219342,2025-04-01,23001,NG030013,33.0,52.3333,40.6788,179.5152,110.4939,236.8485,176.1283,125.5141,157.2819,132.6228,prelim
1219343,2025-04-11,23001,NG030013,33.0,65.7879,43.4556,188.2121,120.9081,301.6364,217.0303,146.0883,151.4132,137.2664,prelim


In [113]:
harvest= pd.read_csv("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed/harvested_long.xlsx")
physical= pd.read_csv("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed/physical_long.xlsx")
yield_data= pd.read_csv("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed/yield_long.xlsx")
production= pd.read_csv("C:/Users/USER/Downloads/Hackathon/AgricConnect-PHL/dataset/harvest_data/processed/production_long.xlsx")